In [2]:
# 本代码由可视化策略环境自动生成 2021年8月22日10:32
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m2_run_bigquant_run(input_1, start_date,end_date):
    # 示例代码如下。在这里编写您的代码
    ins=input_1.read_pickle()['instruments']
    start_date=input_1.read_pickle()['start_date']
    end_date=input_1.read_pickle()['end_date']
    df=DataSource('bar1d_CN_FUND').read(ins)
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m2_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m4_initialize_bigquant_run(context):
    #记录策略运行天数
    context.index = 0
    #获取指标数据
    context.history_ds = context.options['data'].read_df()

# 回测引擎：每日数据处理函数，每天执行一次
def m4_handle_data_bigquant_run(context, data):
    today = data.current_dt.strftime('%Y-%m-%d')
    signal_today=context.history_ds[context.history_ds.date==today]
        
    for instr in context.instruments:
        # 将标的转化为equity格式
        sid = context.symbol(instr)
        try:
            # 短周期均线值
            short_mavg =signal_today['mean(close,5)'].values[0]
            # 长周期均线值
            long_mavg = signal_today['mean(close,10)'].values[0]
        except:
            return
        # 账户持仓
        cur_pos = context.portfolio.positions[sid].amount
        # 策略逻辑部分
        # 空仓状态下，短周期均线上穿（大于）长周期均线形成金叉，买入股票，且该股票可以交易
        # 持仓状态下，短周期均线下穿（小于）长周期均线形成死叉，卖出股票，且该股票可以交易
        if short_mavg > long_mavg and cur_pos == 0:
            context.order_target_percent(sid, 1)
        elif short_mavg < long_mavg and cur_pos > 0:
            context.order_target_percent(sid, 0)
# 回测引擎：准备数据，只执行一次
def m4_prepare_bigquant_run(context):
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m4_before_trading_start_bigquant_run(context, data):
    pass


m3 = M.input_features.v1(
    features="""
# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
#5日均线
mean(close,5)
#10日均线
mean(close,10)""",
    m_cached=False
)

m1 = M.instruments.v2(
    start_date='2013-01-16',
    end_date='2018-01-16',
    market='CN_FUND',
    instrument_list='510330.HOF',
    max_count=0
)

m2 = M.cached.v3(
    input_1=m1.data,
    run=m2_run_bigquant_run,
    post_run=m2_post_run_bigquant_run,
    input_ports='',
    params='{\'start\':\'2018-08-29\'}',
    output_ports=''
)

m5 = M.derived_feature_extractor.v3(
    input_data=m2.data_1,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m7 = M.dropnan.v2(
    input_data=m5.data
)

m4 = M.trade.v4(
    instruments=m1.data,
    options_data=m5.data,
    history_ds=m5.data,
    start_date='',
    end_date='',
    initialize=m4_initialize_bigquant_run,
    handle_data=m4_handle_data_bigquant_run,
    prepare=m4_prepare_bigquant_run,
    before_trading_start=m4_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-08-22 10:31:44.299862] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-08-22 10:31:44.328503] INFO: moduleinvoker: input_features.v1 运行完成[0.028652s].

[2021-08-22 10:31:44.335113] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-08-22 10:31:44.364379] INFO: moduleinvoker: instruments.v2 运行完成[0.029257s].

[2021-08-22 10:31:44.374612] INFO: moduleinvoker: cached.v3 开始运行..

...

[2021-08-22 10:31:56.416750] INFO: Performance: Simulated 1218 trading days out of 1218.

[2021-08-22 10:31:56.418333] INFO: Performance: first open: 2013-01-16 09:30:00+00:00

[2021-08-22 10:31:56.419500] INFO: Performance: last close: 2018-01-16 15:00:00+00:00

[2021-08-22 10:31:58.850940] INFO: moduleinvoker: backtest.v8 运行完成[8.093563s].

[2021-08-22 10:31:58.852437] INFO: moduleinvoker: trade.v4 运行完成[9.88337s].